In [1]:
from Modules import Ingenierie, data, simple_visualisation, advanced_visualisation, widget,data_cleaning
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.calibration import CalibrationDisplay, calibration_curve
import requests
import json
import math
import numpy as np


# Chargement des données et création du dataframe

In [2]:
year_list = range(2016,2024)


In [3]:
#all_data=data.load_data(year_list)
data_cleaning.create_dataframes(year_list, data.load_data(year_list))

Loading data for season 2016


KeyboardInterrupt: 

## Chargement des dataframe correspondant aux années désirées

In [4]:
dataframes =data_cleaning.load_dataframes(year_list)
dataframes[2016].head(70)

,ID,Sort Order,Time,Period,Event ID,Team,Type,Coordinates,Shooter ID,Goalie ID,Shot Type,Empty Net,Force Type,Home
0,2016020001,16,01:11,1,8,Maple Leafs,shot-on-goal,"(-77, 5)",Mitch,Craig,wrist,False,Even Strength,False
1,2016020001,38,02:53,1,11,Senators,shot-on-goal,"(86, 13)",Chris,Frederik,wrist,False,Even Strength,True
2,2016020001,53,04:01,1,15,Senators,shot-on-goal,"(23, -38)",Cody,Frederik,wrist,False,Even Strength,True
3,2016020001,60,04:46,1,16,Senators,shot-on-goal,"(33, -15)",Erik,Frederik,slap,False,Even Strength,True
4,2016020001,86,06:46,1,24,Maple Leafs,shot-on-goal,"(-34, 28)",Martin,Craig,wrist,False,Even Strength,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2016020001,724,18:49,3,865,Senators,shot-on-goal,"(49, -13)",Mark,Frederik,snap,False,Even Strength,True
66,2016020001,726,19:09,3,867,Senators,shot-on-goal,"(43, -18)",Cody,Frederik,slap,False,Even Strength,True
67,2016020001,748,00:37,4,876,Senators,goal,"(-49, 5)",Kyle,Frederik,snap,False,Even Strength,True
68,2016020002,7,00:12,1,7,Blues,shot-on-goal,"(64, -16)",Alexander,Corey,snap,False,Even Strength,False


In [5]:
from Modules import new_data_cleaning
df = dataframes[2016]
df_final = new_data_cleaning.transform_dataframe(df)
print (df_final)

'''
# Fonction pour convertir le temps en secondes
def time_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

# Conversion du temps en secondes
df['Game seconds'] = df['Time'].apply(time_to_seconds)

# Renommer la colonne `Period` en `Game period`
df['Game period'] = df['Period']

# Séparation des coordonnées en colonnes `x_coord` et `y_coord`
df[['x_coord', 'y_coord']] = df['Coordinates'].str.extract(r'\((-?\d+),\s*(-?\d+)\)').astype(float)

# Calculer la distance de tir
def calculate_shot_distance(coord_str):
    if not coord_str or coord_str == 'None':  # Vérifie si la chaîne est vide ou invalide
        return None
    try:
        x, y = map(float, coord_str.strip('()').split(','))
        goal_x = 89 if x > 0 else -89
        return math.sqrt((x - goal_x) ** 2 + y ** 2)
    except Exception:
        return None  # Retourner None en cas d'erreur

df['Shot distance'] = df['Coordinates'].apply(lambda coord: calculate_shot_distance(coord))

# Calculer l'angle de tir
def calculate_shot_angle(coord_str):
    if not coord_str or coord_str == 'None':  # Vérifie si la chaîne est vide ou invalide
        return None
    try:
        x, y = map(float, coord_str.strip('()').split(','))
        goal_x = 89 if x > 0 else -89
        angle_radians = math.atan2(y, (goal_x - x))
        return math.degrees(angle_radians)
    except Exception:
        return None  # Retourner None en cas d'erreur

df['Shot angle'] = df['Coordinates'].apply(lambda coord: calculate_shot_angle(coord))

# Renommer `Shot Type` en `Shot type`
df['Shot type'] = df['Shot Type']

# Dernier type d'événement (Last event type)
df['Last event type'] = df['Type'].shift(1)

# Coordonnées du dernier événement
df['Last x_coord'] = df['x_coord'].shift(1)
df['Last y_coord'] = df['y_coord'].shift(1)

# Temps écoulé depuis le dernier événement
df['Time since last event'] = df['Game seconds'] - df['Game seconds'].shift(1)

# Calculer la distance depuis le dernier événement
def calculate_distance(x1, y1, x2, y2):
    if any(pd.isna([x1, y1, x2, y2])):  # Si une des coordonnées est NaN, retourner NaN
        return np.nan
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

df['Distance from last event'] = df.apply(
    lambda row: calculate_distance(row['x_coord'], row['y_coord'], row['Last x_coord'], row['Last y_coord']), axis=1
)

# Créer le DataFrame final avec toutes les colonnes nécessaires
df_final = df[[
    'ID', 'Game seconds', 'Game period', 'x_coord', 'y_coord',
    'Shot distance', 'Shot angle', 'Shot type',
    'Last event type', 'Last x_coord', 'Last y_coord',
    'Time since last event', 'Distance from last event'
]]

# Afficher le DataFrame final
print(df_final)'''


               ID  Game seconds  Game period  x_coord  y_coord  Shot distance  \
0      2016020001            71            1    -77.0      5.0      13.000000   
1      2016020001           173            1     86.0     13.0      13.341664   
2      2016020001           241            1     23.0    -38.0      76.157731   
3      2016020001           286            1     33.0    -15.0      57.974132   
4      2016020001           406            1    -34.0     28.0      61.717096   
...           ...           ...          ...      ...      ...            ...   
80394  2016030416           617            3     60.0      2.0      29.068884   
80395  2016030416           694            3    -87.0    -12.0      12.165525   
80396  2016030416           944            3     86.0      5.0       5.830952   
80397  2016030416          1105            3    -86.0      5.0       5.830952   
80398  2016030416          1186            3    -78.0      0.0      11.000000   

       Shot angle      Type

"\n# Fonction pour convertir le temps en secondes\ndef time_to_seconds(time_str):\n    minutes, seconds = map(int, time_str.split(':'))\n    return minutes * 60 + seconds\n\n# Conversion du temps en secondes\ndf['Game seconds'] = df['Time'].apply(time_to_seconds)\n\n# Renommer la colonne `Period` en `Game period`\ndf['Game period'] = df['Period']\n\n# Séparation des coordonnées en colonnes `x_coord` et `y_coord`\ndf[['x_coord', 'y_coord']] = df['Coordinates'].str.extract(r'\\((-?\\d+),\\s*(-?\\d+)\\)').astype(float)\n\n# Calculer la distance de tir\ndef calculate_shot_distance(coord_str):\n    if not coord_str or coord_str == 'None':  # Vérifie si la chaîne est vide ou invalide\n        return None\n    try:\n        x, y = map(float, coord_str.strip('()').split(','))\n        goal_x = 89 if x > 0 else -89\n        return math.sqrt((x - goal_x) ** 2 + y ** 2)\n    except Exception:\n        return None  # Retourner None en cas d'erreur\n\ndf['Shot distance'] = df['Coordinates'].apply(l

In [ ]:


df_ad = new_data_cleaning.add_advanced_features(df_final)
df_ad.head(80)


c:\Users\vince\OneDrive\Documents\GitHub\IFT6358\Modules\new_data_cleaning.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prev_event_type'] = df['Type'].shift(1)
c:\Users\vince\OneDrive\Documents\GitHub\IFT6358\Modules\new_data_cleaning.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prev_x_coord'] = df['x_coord'].shift(1)
c:\Users\vince\OneDrive\Documents\GitHub\IFT6358\Modules\new_data_cleaning.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

,ID,Game seconds,Game period,x_coord,y_coord,Shot distance,Shot angle,Type,Last event type,Last x_coord,...,prev_x_coord,prev_y_coord,prev_shot_angle,prev_game_seconds,prev_period,Rebound,Angle change,time_since_last_event,distance_from_last_event,Speed
0,2016020001,71,1,-77.0,5.0,13.000000,157.380135,wrist,None,NaN,...,NaN,NaN,NaN,NaN,NaN,False,0,0.0,0.000000,0.000000
1,2016020001,173,1,86.0,13.0,13.341664,77.005383,wrist,wrist,-77.0,...,-77.0,5.0,157.380135,71.0,1.0,False,0,102.0,163.196201,1.599963
2,2016020001,241,1,23.0,-38.0,76.157731,-29.931512,wrist,wrist,86.0,...,86.0,13.0,77.005383,173.0,1.0,False,0,68.0,81.055537,1.191993
3,2016020001,286,1,33.0,-15.0,57.974132,-14.995079,slap,wrist,23.0,...,23.0,-38.0,-29.931512,241.0,1.0,False,0,45.0,25.079872,0.557330
4,2016020001,406,1,-34.0,28.0,61.717096,153.019769,wrist,slap,33.0,...,33.0,-15.0,-14.995079,286.0,1.0,False,0,120.0,79.611557,0.663430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2016020002,325,1,74.0,-34.0,37.161808,-66.194056,slap,wrist,-79.0,...,-79.0,-3.0,-163.300756,297.0,1.0,False,0,28.0,156.108936,5.575319
76,2016020002,478,1,63.0,-3.0,26.172505,-6.581945,wrist,slap,74.0,...,74.0,-34.0,-66.194056,325.0,1.0,False,0,153.0,32.893768,0.214992
77,2016020002,510,1,72.0,3.0,17.262677,10.007980,backhand,wrist,63.0,...,63.0,-3.0,-6.581945,478.0,1.0,False,0,32.0,10.816654,0.338020
78,2016020002,594,1,-58.0,21.0,37.443290,145.885527,backhand,backhand,72.0,...,72.0,3.0,10.007980,510.0,1.0,False,0,84.0,131.240238,1.562384
